# Выбор локации для скважины

Допустим, мы работаем в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Построим модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# Описание данных

- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

# Условия задачи

- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.
- Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

## Загрузка и подготовка данных

In [1]:
#Импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import scipy.stats as st
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
#Откроем файлы
df_zero = pd.read_csv('/datasets/geo_data_0.csv')
df_first = pd.read_csv('/datasets/geo_data_1.csv')
df_second = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
#Изучим первые 10 строк
df_zero.head(10)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


In [4]:
#Изучим первые 10 строк
df_first.head(10)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
5,HHckp,-3.327590,-2.205276,3.003647,84.038886
6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147
7,muH9x,4.234715,-0.001354,2.004588,53.906522
8,YiRkx,13.355129,-0.332068,4.998647,134.766305
9,jG6Gi,1.069227,-11.025667,4.997844,137.945408


In [5]:
#Изучим первые 10 строк
df_second.head(10)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
5,LzZXx,-0.758092,0.710691,2.585887,90.222465
6,WBHRv,-0.574891,0.317727,1.773745,45.641478
7,XO8fn,-1.906649,-2.458350,-0.177097,72.480640
8,ybmQ5,1.776292,-0.279356,3.004156,106.616832
9,OilcN,-1.214452,-0.439314,5.922514,52.954532


In [6]:
#Посмотрим общую информацию
df_zero.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
#Посмотрим общую информацию
df_first.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
#Посмотрим общую информацию
df_second.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [9]:
#Проверим количество дубликатов
df_zero.duplicated().sum()

0

In [10]:
#Проверим количество дубликатов
df_first.duplicated().sum()

0

In [11]:
#Проверим количество дубликатов
df_second.duplicated().sum()

0

In [12]:
#Разделим данные на признаки
features_names = ['f0', 'f1', 'f2']
target_name = ['product']
features_zero = df_zero[features_names]
target_zero = df_zero[target_name]
features_first = df_first[features_names]
target_first = df_first[target_name]
features_second = df_second[features_names]
target_second = df_second[target_name]

### Вывод

С данными всё в порядке, пропусков и дубликатов не обнаружено. Произошло разделение на признаки. Сказать что-то конкретнее весьма затруднительно, поскольку нет точной информации о том, что означают сами данные в файлах.

## Обучение и проверка модели

Обучим и проверим модель для каждого региона:

- Разобьём данные на обучающую и валидационную выборки в соотношении 75:25.
- Обучим модель и сделаем предсказания на валидационной выборке.
- Сохраним предсказания и правильные ответы на валидационной выборке.
- Напечатаем на экране средний запас предсказанного сырья и RMSE модели.
- Проанализируем результаты.

In [13]:
#Напишем функцию для обучения модели и расчёта среднего запаса предсказанного сырья и RMSE модели
def model_valid(features, target):
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, 
                                                                                  random_state=12345)
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    predictions_valid_mean = predictions_valid.mean()
    rmse = mean_squared_error(target_valid, predictions_valid) ** 0.5
    print('Средний запас предсказанного сырья:', predictions_valid_mean)
    print('RMSE модели линейной регрессии на валидационной выборке:', rmse)
    return target_valid, predictions_valid

In [14]:
#Проверим для 0 региона
target_zero_valid, predictions_zero = model_valid(features_zero, target_zero)

Средний запас предсказанного сырья: 92.59256778438035
RMSE модели линейной регрессии на валидационной выборке: 37.5794217150813


In [15]:
#Проверим для 1 региона
target_first_valid, predictions_first = model_valid(features_first, target_first)

Средний запас предсказанного сырья: 68.728546895446
RMSE модели линейной регрессии на валидационной выборке: 0.893099286775617


In [16]:
#Проверим для 2 региона
target_second_valid, predictions_second = model_valid(features_second, target_second)

Средний запас предсказанного сырья: 94.96504596800489
RMSE модели линейной регрессии на валидационной выборке: 40.02970873393434


### Вывод

Регион 0:

- Средний запас предсказанного сырья: 92.59256778438035
- RMSE модели линейной регрессии на валидационной выборке: 37.5794217150813

Регион 1:

- Средний запас предсказанного сырья: 68.728546895446
- RMSE модели линейной регрессии на валидационной выборке: 0.893099286775617

Регион 2:

- Средний запас предсказанного сырья: 94.96504596800489
- RMSE модели линейной регрессии на валидационной выборке: 40.02970873393434

В регионах 0 и 2 достаточно высокий средний запас предсказанного сырья (92.6 и 95). Однако RMSE для данных регионов также высок (37.6 и 40). Это говорит о неоднозначности показателя, неточности модели регрессии.

В регионе 1 средний запас предсказанного сырья составляет 68.7. Однако RMSE в данном регионе равен 0.9. Это говорит о точности предсказаний и качестве построенной модели.

## Подготовка к расчёту прибыли

Подготовим к расчёту прибыли:

- Все ключевые значения для расчётов сохраним в отдельных переменных.
- Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе.
- Напишем выводы по этапу подготовки расчёта прибыли.

In [17]:
#По условию задачи сделаем обозначения

BUDGET = 10 ** 10 #Бюджет на разработку скважин в регионе — 10 млрд рублей.

TOP_WELLS = 200 #При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших 
WELLS = 500 #для разработки.

PRODUCT = 4.5 * 10 ** 5 #один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 
#450 тыс. рублей, поскольку объём указан в тысячах баррелей.

In [18]:
#Напишем формулу для расчёта достаточного объёма сырья для безубыточной разработки новой скважины
min_product = BUDGET / (TOP_WELLS * PRODUCT)
print('Достаточный объём сырья для безубыточной разработки новой скважины:', min_product)

Достаточный объём сырья для безубыточной разработки новой скважины: 111.11111111111111


### Вывод

- Минимальный объём сырья для безубыточной разработки новой скважины равен 111.11. Данный показатель рассчитан при условии выбора 200 лучших скважин из 500 в выборке.
- Полученное значение превышает средние запасы предсказанного сырья на всех регионах.

## Расчёт прибыли

Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:

- Выберем скважины с максимальными значениями предсказаний.
- Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям.
- Рассчитаем прибыль для полученного объёма сырья.

In [19]:
#Преобразуем в pd.Series()
target_zero_valid = pd.Series(target_zero_valid['product']).reset_index(drop=True)
target_first_valid = pd.Series(target_first_valid['product']).reset_index(drop=True)
target_second_valid = pd.Series(target_second_valid['product']).reset_index(drop=True)

In [20]:
#Преобразуем в pd.Series()
predictions_zero = pd.Series(*predictions_zero.reshape(1,-1))
predictions_first = pd.Series(*predictions_first.reshape(1,-1))
predictions_second = pd.Series(*predictions_second.reshape(1,-1))

In [21]:
#Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели
def profit(target, predictions):
    sample_predictions = predictions.sample(n=WELLS, replace=False, random_state=12345)
    top_predictions = sample_predictions.sort_values(ascending=False)
    top_targets = target[top_predictions.index][:TOP_WELLS]
    return sum(top_targets) * PRODUCT - BUDGET

In [22]:
#Проверим прибыль для 0 региона
round(profit(target_zero_valid, predictions_zero), 2)

679068857.89

In [23]:
#Проверим прибыль для 1 региона
round(profit(target_first_valid, predictions_first), 2)

779479884.18

In [24]:
#Проверим прибыль для 2 региона
round(profit(target_second_valid, predictions_second), 2)

439990143.02

### Вывод

Была рассчитана прибыль по выбранным скважинам и предсказаниям модели:
- 0 регион - 679068857.89
- 1 регион - 779479884.18
- 2 регион - 439990143.02

## Расчёт рисков

Посчитаем риски и прибыль для каждого региона:
- Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
- Найдём среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
- Напишем выводы: предложим регион для разработки скважин и обоснуем выбор.

In [28]:
#Напишем функцию для расчёта рисков, прибыли и доверительных интервалов для каждого региона
state = np.random.RandomState(12345)
def risk_income(predictions, target):
    values = []
    for i in range(1000):
        target_sample = target.sample(n=WELLS, replace=True, random_state=state)
        predictions_sample = predictions[target_sample.index]
        values.append(profit(predictions_sample, target_sample))
        
    values = pd.Series(values)    
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    values_mean = values.mean()
    risk = st.percentileofscore(values, 0)
    print('Средняя прибыль:', values.mean())
    print('Риск:', risk)
 
    return lower, upper

In [29]:
risk_income(target_zero_valid, predictions_zero)

Средняя прибыль: 425938526.9105928
Риск: 6.0


(-102090094.83793645, 947976353.3583689)

In [30]:
risk_income(target_first_valid, predictions_first)

Средняя прибыль: 518259493.6973477
Риск: 0.3


(128123231.43310332, 953612982.0669409)

In [31]:
risk_income(target_second_valid, predictions_second)

Средняя прибыль: 420194005.34405047
Риск: 6.2


(-115852609.1600059, 989629939.8445663)

### Вывод

Для 0 региона:
- Средняя прибыль: 403017767.0801378
- Риск: 6.5 %

Доверительный интервал: (-127782657.57185954, 919876690.3573787)

Для 1 региона:

- Средняя прибыль: 515918291.97759277
- Риск: 0.5 %

Доверительный интервал: (100898364.46347028, 917963147.6971768)

Для 2 региона:

- Средняя прибыль: 413931776.94860935
- Риск: 7.4 %

Доверительный интервал:(-120725658.55790772, 998094387.7986639)

Рекомендуется брать 1 регион для разработки скважин, так как там наименьший риск и доверительный интервал не имеет отрицательных значений.

## Общий вывод

**Для 0 региона:**

- Средний запас предсказанного сырья: 92.59256778438035
- RMSE модели линейной регрессии на валидационной выборке: 37.5794217150813
- Прибыль по выбранным скважинам и предсказаниям модели: 679068857.89

*Техника BootStrap:*

- Средняя прибыль: 403017767.0801378
- Риск: 6.5 %
- Доверительный интервал: (-127782657.57185954, 919876690.3573787)

**Для 1 региона:**

- Средний запас предсказанного сырья: 68.728546895446
- RMSE модели линейной регрессии на валидационной выборке: 0.893099286775617
- Прибыль по выбранным скважинам и предсказаниям модели: 779479884.18

*Техника BootStrap:*

- Средняя прибыль: 515918291.97759277
- Риск: 0.5 %
- Доверительный интервал: (100898364.46347028, 917963147.6971768)

**Для 2 региона:**

- Средний запас предсказанного сырья: 94.96504596800489
- RMSE модели линейной регрессии на валидационной выборке: 40.02970873393434
- Прибыль по выбранным скважинам и предсказаниям модели:439990143.02

*Техника BootStrap:*

- Средняя прибыль: 413931776.94860935
- Риск: 7.4 %
- Доверительный интервал:(-120725658.55790772, 998094387.7986639)

В регионах 0 и 2 достаточно высокий средний запас предсказанного сырья (92.6 и 95). Однако RMSE для данных регионов также высок (37.6 и 40). Это говорит о неоднозначности показателя, неточности модели регрессии.

В регионе 1 средний запас предсказанного сырья составляет 68.7. Однако RMSE в данном регионе равен 0.9. Это говорит о точности предсказаний и качестве построенной модели.

**ДЛЯ РАЗРАБОТКИ СКВАЖИН РЕКОМЕНДУЕТСЯ БРАТЬ 1 РЕГИОН.**